In [ ]:
# Physiological Recording Classification: Healthy vs Impaired
# Simple Random Forest classifier with patient-based splits

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


import warnings
warnings.filterwarnings('ignore')

# Set random state for reproducibility
RANDOM_STATE = 123
np.random.seed(RANDOM_STATE)


In [ ]:
# Load the normalized data tables
print("Loading normalized data tables...")

# Load PSD and SNR normalized data
psd_data = pd.read_csv('detailed_psd_normalized_table.csv')
snr_data = pd.read_csv('detailed_snrs_normalized_table.csv')

print(f"PSD data shape: {psd_data.shape}")
print(f"SNR data shape: {snr_data.shape}")
print(f"\nLabel distribution in PSD data:")
print(psd_data['Arm Type'].value_counts())


In [ ]:
# Combine datasets and prepare features
print("Combining datasets...")

# Merge PSD and SNR data on Patient, Recording, and Arm Type
combined_data = pd.merge(
    psd_data, 
    snr_data, 
    on=['Patient', 'Recording', 'Arm Type'],
    how='inner'
)

print(f"Combined data shape: {combined_data.shape}")
print(f"Combined label distribution:")
print(combined_data['Arm Type'].value_counts())

# Prepare features and labels
feature_cols = [col for col in combined_data.columns 
               if col not in ['Patient', 'Recording', 'Arm Type']]

X = combined_data[feature_cols].values
y = combined_data['Arm Type'].values

print(f"\nFeature matrix shape: {X.shape}")
print(f"Number of features: {len(feature_cols)}")
print(f"PSD features: {len([c for c in feature_cols if 'PSD' in c])}")
print(f"SNR features: {len([c for c in feature_cols if 'SNR' in c])}")


In [ ]:
# Patient-based train/test split function
def patient_based_split(combined_data, X, y, test_size=0.15, random_state=RANDOM_STATE):
    """
    Split data ensuring no patient appears in both train and test sets.
    """
    np.random.seed(random_state)
    
    # Get patient recording counts
    patient_data = combined_data.groupby('Patient').agg({
        'Recording': 'count',
        'Arm Type': lambda x: list(x)
    }).rename(columns={'Recording': 'num_recordings', 'Arm Type': 'arm_types'})
    
    # Calculate target test size
    total_recordings = len(combined_data)
    target_test_recordings = int(total_recordings * test_size)
    
    # Select patients for test set using greedy approach
    patients_by_size = patient_data.sort_values('num_recordings', ascending=False)
    remaining_patients = list(patients_by_size.index)
    
    best_test_patients = []
    best_diff = float('inf')
    
    # Try multiple random combinations to find best split
    for _ in range(100):
        np.random.shuffle(remaining_patients)
        current_test_patients = []
        current_count = 0
        
        for patient in remaining_patients:
            patient_recordings = patient_data.loc[patient, 'num_recordings']
            if current_count + patient_recordings <= target_test_recordings + 5:
                current_test_patients.append(patient)
                current_count += patient_recordings
                
                if abs(current_count - target_test_recordings) < abs(best_diff):
                    best_test_patients = current_test_patients.copy()
                    best_diff = current_count - target_test_recordings
    
    test_patients = best_test_patients
    test_recordings_count = sum(patient_data.loc[p, 'num_recordings'] for p in test_patients)
    
    print(f"Selected {len(test_patients)} patients for test set")
    print(f"Test recordings: {test_recordings_count} ({test_recordings_count/total_recordings*100:.1f}%)")
    
    # Create train/test masks
    test_mask = combined_data['Patient'].isin(test_patients)
    train_mask = ~test_mask
    
    # Split the data
    X_train = X[train_mask]
    X_test = X[test_mask]
    y_train = y[train_mask]
    y_test = y[test_mask]
    
    # Verify no patient overlap
    train_patients = set(combined_data[train_mask]['Patient'].unique())
    test_patients_actual = set(combined_data[test_mask]['Patient'].unique())
    overlap = train_patients.intersection(test_patients_actual)
    
    if overlap:
        print(f"WARNING: Patient overlap detected: {overlap}")
    else:
        print("✓ No patient overlap between train and test sets")
    
    return X_train, X_test, y_train, y_test

# Perform the split
print("Performing patient-based train/test split...")
X_train, X_test, y_train, y_test = patient_based_split(combined_data, X, y)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Training label distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  {label}: {count} ({count/len(y_train)*100:.1f}%)")


In [ ]:
# Model creation function
def create_rf_model():
    """Create a RandomForestClassifier with consistent parameters."""
    return RandomForestClassifier(
        n_estimators=100,
        random_state=RANDOM_STATE,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt'
    )

# Scale features and train Random Forest
print("Scaling features and training Random Forest...")

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train Random Forest
rf_model = create_rf_model()

# Train the model
rf_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_scaled)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {test_accuracy:.3f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
# Cross-validation with patient-based splits
def patient_based_cv_split(combined_data, n_splits=5, random_state=RANDOM_STATE):
    """
    Create cross-validation splits ensuring no patient appears in multiple folds.
    """
    np.random.seed(random_state)
    
    # Get unique patients
    unique_patients = combined_data['Patient'].unique()
    np.random.shuffle(unique_patients)
    
    # Split patients into folds
    fold_size = len(unique_patients) // n_splits
    patient_folds = []
    
    for i in range(n_splits):
        start_idx = i * fold_size
        if i == n_splits - 1:  # Last fold gets remaining patients
            end_idx = len(unique_patients)
        else:
            end_idx = (i + 1) * fold_size
        patient_folds.append(unique_patients[start_idx:end_idx])
    
    # Create train/validation indices for each fold
    cv_splits = []
    for i in range(n_splits):
        val_patients = patient_folds[i]
        val_mask = combined_data['Patient'].isin(val_patients)
        train_mask = ~val_mask
        
        train_indices = combined_data[train_mask].index.values
        val_indices = combined_data[val_mask].index.values
        
        cv_splits.append((train_indices, val_indices))
    
    return cv_splits

print("Performing cross-validation with patient-based splits...")

# Create patient-based CV splits
cv_splits = patient_based_cv_split(combined_data, n_splits=5)

# Perform cross-validation
cv_scores = []
for fold, (train_idx, val_idx) in enumerate(cv_splits):
    # Get train and validation data for this fold
    X_fold_train = X[train_idx]
    X_fold_val = X[val_idx]
    y_fold_train = y[train_idx]
    y_fold_val = y[val_idx]
    
    # Scale features
    fold_scaler = StandardScaler()
    X_fold_train_scaled = fold_scaler.fit_transform(X_fold_train)
    X_fold_val_scaled = fold_scaler.transform(X_fold_val)
    
    # Train model
    fold_model = create_rf_model()
    fold_model.fit(X_fold_train_scaled, y_fold_train)
    
    # Evaluate
    fold_pred = fold_model.predict(X_fold_val_scaled)
    fold_accuracy = accuracy_score(y_fold_val, fold_pred)
    cv_scores.append(fold_accuracy)
    
    print(f"Fold {fold + 1}: {fold_accuracy:.3f}")

cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f"\nCross-Validation Results:")
print(f"Mean CV Accuracy: {cv_mean:.3f} ± {cv_std:.3f}")
print(f"Individual fold scores: {[f'{score:.3f}' for score in cv_scores]}")


In [ ]:
# Plot confusion matrix for test set
print("Plotting confusion matrix...")

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['Healthy', 'Impaired'], 
           yticklabels=['Healthy', 'Impaired'])
plt.title(f'Confusion Matrix - Random Forest\nTest Accuracy: {test_accuracy:.3f}')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

# Print summary
print(f"\nSUMMARY:")
print(f"========")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Cross-Validation Accuracy: {cv_mean:.3f} ± {cv_std:.3f}")
print(f"Training samples: {len(y_train)}")
print(f"Test samples: {len(y_test)}")
print(f"Features used: {X.shape[1]} (PSD + SNR normalized features)")
